# Installs and Imports

## Imports

In [3]:
import logging
import sys

from neo4j import GraphDatabase, basic_auth
from neo4j.exceptions import ServiceUnavailable

# Variables for database connection

In [4]:
# MDB sandbox
url = "bolt://localhost:7687" #"<URL for database>"
user = "neo4j" #"<Username for database>"
password = "noble-use-dairy" #"<Password for database>"
driver = GraphDatabase.driver(url, auth=(user, password))

# Functions

## Helper Functions

### Generate unique nanoid

In [5]:
# generate nanoid
import random
import string

def generate_nanoid():
    valid_chars = string.ascii_letters + string.digits
    nanoid = ''.join(random.choice(valid_chars) for i in range(6))
    return nanoid

In [6]:
def generate_unique_nanoid(tx):
    nanoid = generate_nanoid()
    result = tx.run("MATCH (n {nanoid: $nanoid}) "
                    "RETURN n.nanoid", nanoid=nanoid)
    if not [record["n.nanoid"] for record in result]:
        #print(nanoid)
        return nanoid
    else:
        generate_unique_nanoid(tx)

### Create Concept

In [7]:
# create concept
def create_concept(tx, concept_nanoid):    
        tx.run("MERGE (n:concept {nanoid: $concept_nanoid})", 
                concept_nanoid=concept_nanoid)
        print(f"Created new Concept with nanoid: {concept_nanoid}")

### Create Predicate

In [8]:
# create Predicate
def create_predicate(tx, predicate_nanoid, predicate_handle):    
        tx.run("MERGE (n:predicate {nanoid: $predicate_nanoid, handle: $predicate_handle})", 
                predicate_nanoid=predicate_nanoid, predicate_handle=predicate_handle)
        print(f"Created new Predicate with nanoid: {predicate_nanoid} and handle: {predicate_handle}")

In [9]:
with driver.session() as session:
    pred_nanoid_1 = session.read_transaction(generate_unique_nanoid)
    pred_handle_1 = "exactMatch"
    session.write_transaction(create_predicate, pred_nanoid_1, pred_handle_1)
driver.close() 

Created new Predicate with nanoid: 2cEzXW and handle: exactMatch


### Link two Concepts to a Predicate

General pattern: (c1:concept)<-[:has_subject]-(p:predicate {handle:“exactMatch”})-[:has_object]->(c2:concept)

In [10]:
def create_subject_relationship(tx, concept_nanoid, predicate_nanoid):
        tx.run("MATCH (c:concept {nanoid: $concept_nanoid}), "
                "(p:predicate {nanoid: $predicate_nanoid}) "
                "MERGE (p)-[:has_subject]->(c)", 
                concept_nanoid=concept_nanoid, 
                predicate_nanoid=predicate_nanoid)
        print(f"Created has_subject relationship between source Predicate: {predicate_nanoid} and destination Concept: {concept_nanoid}")

In [11]:
def create_object_relationship(tx, concept_nanoid, predicate_nanoid):
        tx.run("MATCH (c:concept {nanoid: $concept_nanoid}), "
                "(p:predicate {nanoid: $predicate_nanoid}) "
                "MERGE (p)-[:has_object]->(c)", 
                concept_nanoid=concept_nanoid, 
                predicate_nanoid=predicate_nanoid)
        print(f"Created has_object relationship between source Predicate: {predicate_nanoid} and destination Concept: {concept_nanoid}")

Example: Concepts with nanoid: "4jtzA3" and nanoid: "UWs6Di" both represent terms with value: Lung in the MDB. Let's link them with the predicate we just created.

In [12]:
with driver.session() as session:
    predicate_nanoid = "2cEzXW" #"<predicate_nanoid_from_above>"
    concept_nanoid_1 = "4jtzA3"
    concept_nanoid_2 = "UWs6Di"
    session.write_transaction(create_subject_relationship, concept_nanoid_1, predicate_nanoid)
    session.write_transaction(create_object_relationship, concept_nanoid_2, predicate_nanoid)
driver.close()

Created has_subject relationship between source Predicate: 2cEzXW and destination Concept: 4jtzA3
Created has_object relationship between source Predicate: 2cEzXW and destination Concept: UWs6Di


### Get list of Terms represtented by a Concept

In [13]:
def get_terms(tx, concept_nanoid):
    terms = []
    result = tx.run("MATCH (t:term)-[:represents]->(c:concept {nanoid: $concept_nanoid}) "
                    "RETURN t.nanoid AS term", concept_nanoid=concept_nanoid)
    for record in result:
        terms.append(record["term"])
    return terms

### Detach and delete Predicate with nanoid

In [14]:
def detach_delete_predicate(tx, nanoid):
    tx.run("match (p:predicate {nanoid: $nanoid})"
            "detach delete p", nanoid=nanoid)

In [32]:
with driver.session() as session:
    nanoid_2_del = "isAxe4"
    session.write_transaction(detach_delete_predicate, nanoid_2_del)
driver.close() 

### Detach and delete Concept with nanoid

In [15]:
def detach_delete_concept(tx, nanoid):
    tx.run("match (c:concept {nanoid: $nanoid})"
            "detach delete c", nanoid=nanoid)

### Create terms & represents relationship (adapted from linking_terms to use nanoids)

In [16]:
def create_term_from_nanoid(tx, term_val, term_nanoid):
        tx.run("MERGE (n:term {origin_name: 'NDC', "
                "nanoid: $term_nanoid})", 
                term_val=term_val, term_nanoid=term_nanoid)
        print(f"Created new Term with value: {term_val} and nanoid: {term_nanoid}")

# link term and concept
def create_represents_relationship_from_nanoid(tx, term_nanoid, concept_nanoid):
        tx.run("MATCH (t:term {nanoid: $term_nanoid}), "
                "(c:concept {nanoid: $concept_nanoid}) "
                "MERGE (t)-[r:represents]->(c) ", 
                term_nanoid=term_nanoid,
                concept_nanoid=concept_nanoid)
        print(f"Term with nanoid:{term_nanoid} now represents Concept: {concept_nanoid}")

# Combining Concepts

When two existing Concept nodes are deemed synonymous, there are two primary ways to approach linking them together. The first way to link the synonymous Concepts would be via a Predicate node with the an "exactMatch" handle. This method maintains the exisiting Concept & Term structure while adding to it, allowing queries already in use to continue to work. 

The second way is simply merging the two so they are represented by the same Concept node. With this approach, the Terms linked to each Concept would then be linked to the new merged Concept instead. They could be merged under one of the exisiting Concepts or a new Concept could be created and the old two removed. This method would invalidate existing queries using relevant Concepts & Terms.

In [17]:
# link Concepts via Predicate
def link_concepts_to_predicate(concept_id_1: str, concept_id_2: str, predicate_handle="exactMatch") -> None:
    """
    Links two synonymous Concepts via a Predicate 

    This function takes two synonymous Concept nanoids as input strings and links 
    them via a Predicate node and has_subject and has_object relationships. The 
    predicate_handle parameter is defaulted to "exactMatch" but other similar
    SKOS terms include "relatedMatch" and "closeMatch".
    """
    
    with driver.session() as session:

        # create predicate
        new_predicate_nanoid = session.read_transaction(generate_unique_nanoid)
        session.write_transaction(create_predicate, new_predicate_nanoid, predicate_handle)

        # link concepts to predicate via subject & object relationships
        session.write_transaction(create_subject_relationship, concept_nanoid_1, new_predicate_nanoid)
        session.write_transaction(create_object_relationship, concept_nanoid_2, new_predicate_nanoid)

    driver.close()

In [26]:
link_concepts_to_predicate("4jtzA3", "UWs6Di")

Created new Predicate with nanoid: isAxe4 and handle: exactMatch
Created has_subject relationship between source Predicate: isAxe4 and destination Concept: 4jtzA3
Created has_object relationship between source Predicate: isAxe4 and destination Concept: UWs6Di


In [18]:
# merge Concepts into one
def merge_two_concepts(concept_id_1: str, concept_id_2: str) -> None:
    """
    Combine two synonymous Concepts into a single Concept

    This function takes two synonymous Concept nanoids as input strings and
    merges them into a single Concept.
    """

    with driver.session() as session:
        
        # get list of all terms connected to concept 2
        c2_terms = session.read_transaction(get_terms, concept_id_2)

        # delete concept 2
        session.write_transaction(detach_delete_concept, concept_id_2)

        # connect terms from deleted concept to remaining concept
        for term_id in c2_terms:
            session.write_transaction(create_represents_relationship_from_nanoid, term_id, concept_id_1)

    driver.close()

# Testing

In [61]:
with driver.session() as session:
    id_1 = session.read_transaction(generate_unique_nanoid)
    session.write_transaction(create_concept, id_1)
    id_2 = session.read_transaction(generate_unique_nanoid)   
    session.write_transaction(create_concept, id_2)
driver.close()

Created new Concept with nanoid: emSArC
Created new Concept with nanoid: S0mbPX


In [62]:
with driver.session() as session:
    term_id_1 = session.read_transaction(generate_unique_nanoid)
    session.write_transaction(create_term_from_nanoid, "Nelson", term_id_1)
    term_id_2 = session.read_transaction(generate_unique_nanoid)
    session.write_transaction(create_term_from_nanoid, "Nelly", term_id_2)
    term_id_3 = session.read_transaction(generate_unique_nanoid)
    session.write_transaction(create_term_from_nanoid, "Nelsinghouse", term_id_3)
    session.write_transaction(create_represents_relationship_from_nanoid, term_id_1, "emSArC")
    session.write_transaction(create_represents_relationship_from_nanoid, term_id_2, "emSArC")
    session.write_transaction(create_represents_relationship_from_nanoid, term_id_3, "S0mbPX")
    driver.close()

Created new Term with value: Nelson and nanoid: wZ1yrF
Created new Term with value: Nelly and nanoid: 6TntuU
Created new Term with value: Nelsinghouse and nanoid: klKsMN
Term with nanoid:wZ1yrF now represents Concept: emSArC
Term with nanoid:6TntuU now represents Concept: emSArC
Term with nanoid:klKsMN now represents Concept: S0mbPX


In [63]:
with driver.session() as session:
    concept_1 = "emSArC"
    concept_2 = "S0mbPX"
    terms_1 = session.read_transaction(get_terms, concept_1)
    terms_2 = session.read_transaction(get_terms, concept_2)
    print(f"{terms_1}")
    print(f"{terms_2}")
driver.close()

['6TntuU', 'wZ1yrF']
['klKsMN']


In [64]:
merge_two_concepts("emSArC", "S0mbPX")

Term with nanoid:klKsMN now represents Concept: emSArC
